In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import nltk
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer



In [6]:
con=sqlite3.connect('database.sqlite')
filtered_data=pd.read_sql_query("""select * from reviews where Score!=3""",con)
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [10]:
def  partition(x):
    if x<3:
        return 'negative'
    return 'positive'

actualScore=filtered_data['Score']
positiveNeg=actualScore.map(partition)
filtered_data['Score']=positiveNeg
filtered_data.shape


(525814, 10)

In [14]:
display=pd.read_sql_query("""select * from reviews where Score!=0 AND UserId='AR5J8UI46CURR' order by ProductId""",con)
sorted_data=filtered_data.sort_values(by='ProductId',axis=0,ascending=True,inplace=False)
final=sorted_data.drop_duplicates(subset={'UserId','Time','Text','ProfileName'},keep='first',inplace=False)
final.shape



(364173, 10)

In [15]:
display=pd.read_sql_query("""select * from reviews where Score!=3 AND Id=44737 OR Id=64422""",con)

In [17]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
final.shape

(364171, 10)

In [19]:
final['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

# Pre-Processing

In [21]:
import nltk
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()
print(lemmatizer.lemmatize("Cacti"))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DINESH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


Cacti


In [24]:
print(lemmatizer.lemmatize('delicious'))

delicious


In [34]:
count_vect=CountVectorizer()
final_counts=count_vect.fit_transform(final['Text'].values)


In [60]:
print(final_counts.shape)
print(final.shape)

(364171, 115281)
(364171, 11)


In [38]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [53]:
stop=set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr," ",sentence)
    return cleantext

def cleanpunctuation(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned=re.sub(r'[.|,|(|)|\|/]',r' ',cleaned)
    return cleaned

print(stop)
print('****************')
print(sno.stem('tasty'))

{'hasn', "won't", "shan't", 'me', 'so', 'be', 'but', 'how', 'are', 'ma', 'our', 'on', "mustn't", "hasn't", 'very', "should've", 'a', 'before', 'now', 'isn', 'those', 'yourself', "it's", 'did', 'during', 'will', 'when', 'each', 'wouldn', 'other', 'll', 'who', 'no', 'mustn', "needn't", 'under', 'itself', 'them', "you'll", 'her', 'over', 'doing', 'we', 'your', 'she', "you'd", "didn't", 'is', 'below', 'until', 's', 'had', 'own', 'at', "weren't", 'd', 'my', 'they', 'hadn', "wouldn't", 'couldn', 'hers', 'you', 'weren', 'herself', 'having', 'more', 'doesn', 'aren', "shouldn't", 'y', 'mightn', 'shouldn', 'whom', 'above', 'do', 'their', 'after', 'ours', 'themselves', 'between', 'i', 'didn', 'its', 'does', 'off', 'wasn', 'most', "wasn't", 'which', 'by', 'again', "haven't", "hadn't", 'then', 'for', 'o', 'just', 'all', 'ain', 'if', 'were', 'what', 'there', 'an', 'it', 'too', 'with', 'of', 'nor', "you're", 'this', 'won', 'theirs', 'needn', 'why', 'same', 'through', 'into', "isn't", 've', 'yourselve

In [40]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DINESH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
#Code for imlementing step- by-step preprocessing

i=0

In [47]:
"hello, dinesh; how r u".split()

['hello,', 'dinesh;', 'how', 'r', 'u']

In [48]:
'dinesh'.split()

['dinesh']

In [54]:
i=0
strl=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
a=''
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunctuation(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if(final['Score'].values)[i]=='positive':
                        all_positive_words.append(s)
                    if(final['Score'].values)[i]=='negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
        
    strl=b" ".join(filtered_sentence)    
    
    
    final_string.append(strl)
    i+=1
    



In [57]:
final['CleanedText']=final_string
final.head(3)


conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
final.to_sql('Reviews',conn,schema=None,if_exists='replace')


In [58]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most Common Positive Words ",freq_dist_positive.most_common(20))
print("Most Common Negative Words ",freq_dist_negative.most_common(20))


Most Common Positive Words  [(b'like', 139429), (b'tast', 129047), (b'good', 112766), (b'flavor', 109624), (b'love', 107357), (b'use', 103888), (b'great', 103870), (b'one', 96726), (b'product', 91033), (b'tri', 86791), (b'tea', 83888), (b'coffe', 78814), (b'make', 75107), (b'get', 72125), (b'food', 64802), (b'would', 55568), (b'time', 55264), (b'buy', 54198), (b'realli', 52715), (b'eat', 52004)]
Most Common Negative Words  [(b'tast', 34585), (b'like', 32330), (b'product', 28218), (b'one', 20569), (b'flavor', 19575), (b'would', 17972), (b'tri', 17753), (b'use', 15302), (b'good', 15041), (b'coffe', 14716), (b'get', 13786), (b'buy', 13752), (b'order', 12871), (b'food', 12754), (b'dont', 11877), (b'tea', 11665), (b'even', 11085), (b'box', 10844), (b'amazon', 10073), (b'make', 9840)]


# BoW

In [59]:
count_vect=CountVectorizer(ngram_range=(1,2))
final_bigram_counts=count_vect.fit_transform(final['Text'].values)
final_bigram_counts.shape

(364171, 2910192)

# Tf-Idf

In [62]:
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_vect.fit_transform(final['Text'].values)


In [64]:
final_tf_idf.shape

(364171, 2910192)

In [65]:
features=tf_idf_vect.get_feature_names()
len(features)

2910192

In [66]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

In [67]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [68]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df


top_tfidf=top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [71]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


In [72]:
final["CleanedText"]

138706    b'witti littl book make son laugh loud recit c...
138688    b'grew read sendak book watch realli rosi movi...
138689    b'fun way children learn month year learn poem...
138690    b'great littl book read nice rhythm well good ...
138691    b'book poetri month year goe month cute littl ...
138693    b'charm rhyme book describ circumst eat dont c...
138694    b'set asid least hour day read son point consi...
138695    b'rememb book childhood got kid good rememb ki...
138696    b'great book ador illustr true classic kid lov...
138697    b'book famili favorit read children small orde...
138687    b'get movi sound track sing along carol king g...
138698    b'author wrote wild thing carol king wrote gre...
138700    b'great book perfect condit arriv short amount...
138701    b'ive alway love chicken soup rice late ethel ...
138702    b'book purchas birthday gift year old boy sque...
138703    b'year old daughter brought book home school l...
138704    b'book contain collect twelv s

In [80]:
string = ['Python is a scripting language','Perl is as scripting language','Java is a programming language']
k_vect = TfidfVectorizer()
z = k_vect.fit_transform(string)

In [84]:
print(z.shape)

(3, 8)


In [82]:
z.toarray()

array([[0.        , 0.39148397, 0.        , 0.39148397, 0.        ,
        0.        , 0.66283998, 0.50410689],
       [0.55249005, 0.32630952, 0.        , 0.32630952, 0.55249005,
        0.        , 0.        , 0.42018292],
       [0.        , 0.35959372, 0.6088451 , 0.35959372, 0.        ,
        0.6088451 , 0.        , 0.        ]])

In [83]:
k_vect.get_feature_names()

['as', 'is', 'java', 'language', 'perl', 'programming', 'python', 'scripting']

In [86]:
string = ['Python is a scripting language','Perl is as scripting language','Java is a programming language']
k_vect = TfidfVectorizer(stop_words='english')
z = k_vect.fit_transform(string)
print(z.shape)
print(z.toarray())
print(k_vect.get_feature_names())

(3, 6)
[[0.         0.42544054 0.         0.         0.72033345 0.54783215]
 [0.         0.42544054 0.72033345 0.         0.         0.54783215]
 [0.65249088 0.38537163 0.         0.65249088 0.         0.        ]]
['java', 'language', 'perl', 'programming', 'python', 'scripting']


In [87]:
string = ['Python is a scripting language','Perl is as scripting language','Java is a programming language']
k_vect = CountVectorizer(stop_words='english')
z = k_vect.fit_transform(string)
print(z.shape)
print(z.toarray())
print(k_vect.get_feature_names())

(3, 6)
[[0 1 0 0 1 1]
 [0 1 1 0 0 1]
 [1 1 0 1 0 0]]
['java', 'language', 'perl', 'programming', 'python', 'scripting']


In [88]:
string = ['Python is a scripting language','Perl is as scripting language','Java is a programming language']
k_vect = CountVectorizer()
z = k_vect.fit_transform(string)
print(z.shape)
print(z.toarray())
print(k_vect.get_feature_names())

(3, 8)
[[0 1 0 1 0 0 1 1]
 [1 1 0 1 1 0 0 1]
 [0 1 1 1 0 1 0 0]]
['as', 'is', 'java', 'language', 'perl', 'programming', 'python', 'scripting']


# Word2Vec

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle+


model=KeyedVectors.load_word2vec_format('GoogleNews-vector-Negative300')